### Web Scrapping

In [ ]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [146]:
lang = "bangla"  # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "adh dhaariyat"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-51"  # al faatiha-1, al baqara-2, aal i imraan-3 ...
file = f"{surah.lower()}_{lang.lower()}"

In [147]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

content = HTMLFile.read()

soup = BeautifulSoup(content, 'lxml')

ayah_word = soup.findAll('span', attrs={"class":"single-word"})

word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    #arabic = var1.get_text()
    #    arabic = var1.string
    #except:
    #    arabic = ""
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
df_word_list

,arabic,transliterate,translate_bangla
0,وَٱلذَّـٰرِيَٰتِ,wal-dhariyati,শপথ বিক্ষিপ্তকারীদের (অর্থাৎ বাতাসের)
1,ذَرۡوٗا,dharwan,বিক্ষিপ্ত করার মতো (যা ধুলাবালি)
2,فَٱلۡحَٰمِلَٰتِ,fal-hamilati,অতঃপর বহনকারীর
3,وِقۡرٗا,wiq'ran,ভার (অর্থাৎ মেঘ)
4,فَٱلۡجَٰرِيَٰتِ,fal-jariyati,অতঃপর বয়ে যাওয়া বাতাসের
...,...,...,...
355,كَفَرُواْ,kafaru,অস্বীকার করেছে
356,مِن,min,ব্যাপারে
357,يَوۡمِهِمُ,yawmihimu,তাদের (সেই) দিনের
358,ٱلَّذِي,alladhi,যার


In [148]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge Word All Translate

In [149]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['Adh dhaariyat_Arabic_Transliterate_Translate_Words_Indonesian.xlsx',
 'Adh dhaariyat_Arabic_Transliterate_Translate_Words_Turkish.xlsx',
 'Adh dhaariyat_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'Adh dhaariyat_Arabic_Transliterate_Translate_Words_English.xlsx',
 'Adh dhaariyat_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'Adh dhaariyat_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'Adh dhaariyat_Arabic_Transliterate_Translate_Words_Bangla.xlsx']

In [150]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,وَٱلذَّـٰرِيَٰتِ,wal-dhariyati,By those scattering,قسم ہے ان ہواؤں کی جو گرد اڑانے والی ہیں,क़सम है उन हवाओं की जो बिखेरने वाली हैं,demi yang menerbangkan,শপথ বিক্ষিপ্তকারীদের (অর্থাৎ বাতাসের),kaldıran(rüzgar)lara andolsun,Клянусь рассеивающими
1,ذَرۡوٗا,dharwan,dispersing,گرد اڑانا,उड़ा कर,dengan menerbangkan sekuat-kuatnya,বিক্ষিপ্ত করার মতো (যা ধুলাবালি),savurup,рассеиванием
2,فَٱلۡحَٰمِلَٰتِ,fal-hamilati,And those carrying,پھر اٹھانے والیاں ہیں,फिर उठाने वालियां हैं,maka yang mengandung,অতঃপর বহনকারীর,yüklü (bulut)lara andolsun,и несущими
3,وِقۡرٗا,wiq'ran,a load,بوجھ کو,बोझ को,beban berat,ভার (অর্থাৎ মেঘ),ağır,ношу
4,فَٱلۡجَٰرِيَٰتِ,fal-jariyati,And those sailing,پھر چلنے والیاں ہیں,फिर चलने वालियां हैं,maka yang berlayar,অতঃপর বয়ে যাওয়া বাতাসের,akıp gidenlere andolsun,и плывущими
...,...,...,...,...,...,...,...,...,...
355,كَفَرُواْ,kafaru,disbelieve,جنہوں نے کفر کیا,कुफ़्र किया,kafir,অস্বীকার করেছে,*,стали неверующими
356,مِن,min,from,سے,*,dari,ব্যাপারে,dolayı,от
357,يَوۡمِهِمُ,yawmihimu,their Day,اس دن,उनके उस दिन से,pada hari mereka,তাদের (সেই) দিনের,günlerinden,дня их
358,ٱلَّذِي,alladhi,which,ان کے,जिसका,yang,যার,uyarıldıkları,который


In [151]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)

#### Merge Surah All Translate

In [ ]:
def add_info(df,surah_name, surah_num, index):
    for i in range(len(df)):
        df["surah latin"] = surah_name
        df["surah"] = surah_num
        df.loc[i,"index"] = i+index
    return df

In [ ]:
df_faatir = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Faatir-35/Faatir_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_faatir

In [ ]:
df_faatir = add_info(df_faatir, "Fatir", 35, 56320)
df_faatir = df_faatir.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_faatir

In [ ]:
df_yaseen = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Yaseen-36/Yaseen_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_yaseen

In [ ]:
df_yaseen = add_info(df_yaseen, "Ya Seen", 36, 57102)
df_yaseen = df_yaseen.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_yaseen

In [ ]:
df_saaffaat = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/As Saaffaat-37/As saaffaat_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saaffaat

In [ ]:
df_saaffaat = add_info(df_saaffaat, "As-Saffat", 37, 57836)
df_saaffaat = df_saaffaat.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saaffaat

In [ ]:
df_saad = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Saad-38/Saad_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saad

In [ ]:
df_saad = add_info(df_saad, "Sad", 38, 58705)
df_saad = df_saad.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saad

In [ ]:
df_zumar = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Az Zumar-39/Az zumar_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_zumar

In [ ]:
df_zumar = add_info(df_zumar, "Az-Zumar", 39, 59444)
df_zumar = df_zumar.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_zumar

In [ ]:
df_surah_merge = pd.concat([df_faatir,df_yaseen,df_saaffaat,df_saad,df_zumar], axis=0)
df_surah_merge

In [ ]:
df_surah_merge.to_excel("Faatir_Yaseen_Saaffaat_Saad_Zumar_Surah_Translate_Concat.xlsx", index=False)